In [10]:
# Torch-related imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset

# Scikit-learn-related imports
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score

# Nibabel and Scipy imports (for handling fMRI and image processing)
import nibabel as nib
import scipy.ndimage as ndimage  # For smoothing

# NumPy, Matplotlib, and Seaborn (for data manipulation and visualization)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# OS for file system operations
import os

In [11]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Preprocessing

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = os.path.join('/content/drive', 'My Drive', 'UCR', '2-2024', 'InvCC', 'ADHD200', 'Datasets', 'preprocessed')

root_dir = os.path.join('preprocessed')

# Carpetas para TDC y ADHD
tdc_dir = os.path.join(root_dir, 'TDC')
adhd_dir = os.path.join(root_dir, 'ADHD')

# Para guardar el estado del autoencoder
save_path = os.path.join(root_dir, 'autoencoder.pt')

# Listas para almacenar las rutas de archivos
tdc_file_paths = [os.path.join(tdc_dir, file) for file in os.listdir(tdc_dir) if file.endswith('.nii.gz')]
adhd_file_paths = [os.path.join(adhd_dir, file) for file in os.listdir(adhd_dir) if file.endswith('.nii.gz')]

# Etiquetas correspondientes
tdc_labels = [0] * len(tdc_file_paths)
adhd_labels = [1] * len(adhd_file_paths)

# Combinar rutas de archivos y etiquetas
file_paths = tdc_file_paths + adhd_file_paths
labels = tdc_labels + adhd_labels

In [13]:
class FMRI_Dataset(Dataset):
    def __init__(self, file_paths, labels, max_shape, smoothing_sigma=1):
        self.file_paths = file_paths  # List of paths to the fMRI data files
        self.labels = labels  # Corresponding labels
        self.max_shape = max_shape  # Shape to pad all inputs to (e.g., [1, 53, 64, 46, 512])
        self.smoothing_sigma = smoothing_sigma  # Standard deviation for Gaussian smoothing

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # Load fMRI data using NiBabel
        fmri_img = nib.load(self.file_paths[idx])
        data = fmri_img.get_fdata()  # Extract the fMRI data as a NumPy array

        # Apply smoothing
        data = self.smooth_data(data)

        # Normalize the data
        data = self.normalize_data(data)

        # Convert NumPy array to a PyTorch tensor and add missing dimensions as needed
        data = torch.tensor(data, dtype=torch.float32).unsqueeze(0)

        # Pad the tensor to the specified max_shape
        data_padded = F.pad(data, pad=self.calculate_padding(data.shape), mode='constant', value=0)

        # Ensure that the final shape matches max_shape exactly
        data_padded = data_padded.view(*self.max_shape)

        # Get the label
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return data_padded, label

    def calculate_padding(self, current_shape):
        padding = []
        # Reverse the dimensions to calculate padding from last dimension to the first
        for current_dim, max_dim in zip(reversed(current_shape), reversed(self.max_shape)):
            pad_total = max_dim - current_dim
            padding.append(pad_total // 2)  # pad_left or pad_top, etc.
            padding.append(pad_total - (pad_total // 2))  # pad_right or pad_bottom, etc.
        return padding

    def normalize_data(self, data):
        """Normalize the data to zero mean and unit variance."""
        mean = data.mean()
        std = data.std()
        if std > 0:  # Avoid division by zero
            data = (data - mean) / std
        return data

    def smooth_data(self, data):
        """Apply Gaussian smoothing to the data."""
        return ndimage.gaussian_filter(data, sigma=self.smoothing_sigma)

In [14]:
# Determine the maximum shape across all tensors
max_shape = [1, 53, 64, 46, 512]

# Create the dataset with padded tensors
dataset = FMRI_Dataset(file_paths, labels, max_shape)

sample_data, sample_label = dataset[0]
print(sample_data.shape, sample_label)

torch.Size([1, 53, 64, 46, 512]) tensor(0)


In [15]:
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import torch

# Create StratifiedShuffleSplit object to split into training and test sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Ensure labels are on CPU for compatibility with sklearn
labels_cpu = dataset.labels.cpu().numpy() if isinstance(dataset.labels, torch.Tensor) else dataset.labels

# Split into training and test sets
for train_val_idx, test_idx in sss.split(range(len(dataset)), labels_cpu):
    train_val_set = Subset(dataset, train_val_idx)
    testset = Subset(dataset, test_idx)

# Create another StratifiedShuffleSplit for training and validation sets
sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)  # 0.25 of 0.8 gives 0.2 total for validation

# Access labels for train_val_set
train_val_labels = [labels_cpu[i] for i in train_val_idx]  # Use list comprehension to retrieve labels

# Split train_val_set into train and validation sets
for train_idx, val_idx in sss_val.split(range(len(train_val_set)), train_val_labels):
    trainset = Subset(train_val_set, train_idx)
    valset = Subset(train_val_set, val_idx)

# Create dataloaders
batch_size = 1
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=0)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

class_names = ["TDC", "ADHD"]

# Verify the dataloaders
for batch in trainloader:
    if batch[0] is not None:
        images, labels = batch
        break
        # print(f"Shape of inputs: {images.shape}, Shape of labels: {labels}")

for batch in testloader:
    if batch[0] is not None:
        images, labels = batch
        break
        # print(f"Shape of inputs: {images.shape}, Shape of labels: {labels}")

In [16]:
print(f"Number of samples in valloader: {len(valloader.dataset)}")
print(f"Number of samples in trainloader: {len(trainloader.dataset)}")
print(f"Number of samples in testloader: {len(testloader.dataset)}")

Number of samples in valloader: 561
Number of samples in trainloader: 1680
Number of samples in testloader: 561


# CNN-AE

In [17]:
class CNN_Autoencoder(nn.Module):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(16, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(256, 128, kernel_size=3, stride=2, padding=(1, 1, 1), output_padding=(0, 1, 0)),
            nn.ReLU(),
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=(1, 1, 1), output_padding=(1, 0, 1)),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 16, kernel_size=3, stride=2, padding=(1, 0, 0), output_padding=(0, 0, 1)),
            nn.ReLU(),
            nn.ConvTranspose3d(16, 1, kernel_size=3, stride=2, padding=(1, 0, 0), output_padding=(0, 1, 1)),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = CNN_Autoencoder().to(device)

# Generate random input matching new shape [1, 1, 53, 64] (Batch size 1)
inputs = torch.rand((1, 53, 64, 46)).to(device)  # Example input
output = autoencoder(inputs)
print(output.shape)  # should match the input shape [1, 53, 64, 46]

# Loss and optimizer
criterion = nn.MSELoss()  # Since it's an autoencoder, Mean Squared Error is commonly used
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)


torch.Size([1, 53, 64, 46])


In [18]:
# Number of epochs
epochs = 100

# Track the best validation loss
best_val_loss = float('inf')

for epoch in range(epochs):
    # Training phase
    autoencoder.train()  # Set the model to training mode
    total_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        inputs_reduced = torch.mean(inputs, dim=-1)
        inputs = inputs_reduced

        inputs = inputs.to(device)  # Send to GPU if available

        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)  # Compare reconstruction with original input

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = total_loss / len(trainloader)
    print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {avg_train_loss}")

    # Validation phase
    autoencoder.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation
        for val_inputs, val_labels in valloader:
            val_inputs_reduced = torch.mean(val_inputs, dim=-1)
            val_inputs = val_inputs_reduced.to(device)  # Send validation inputs to GPU if available

            # Forward pass
            val_outputs = autoencoder(val_inputs)
            val_loss = criterion(val_outputs, val_inputs)  # Compare reconstruction with original input
            total_val_loss += val_loss.item()

    # Calculate average validation loss
    avg_val_loss = total_val_loss / len(valloader)
    print(f"Epoch [{epoch+1}/{epochs}], Validation Loss: {avg_val_loss}")

    # Save the model if the validation loss has decreased
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(autoencoder.state_dict(), save_path)
        print(f"Model saved at epoch {epoch+1} with validation loss: {avg_val_loss}")

# Final save after training
torch.save(autoencoder.state_dict(), save_path)

Epoch [1/100], Training Loss: 0.022387138504113648
Epoch [1/100], Validation Loss: 0.015976974350498215
Model saved at epoch 1 with validation loss: 0.015976974350498215
Epoch [2/100], Training Loss: 0.010911609241780492
Epoch [2/100], Validation Loss: 0.009090638543734802
Model saved at epoch 2 with validation loss: 0.009090638543734802
Epoch [3/100], Training Loss: 0.007110771070507499
Epoch [3/100], Validation Loss: 0.011868832764359367
Epoch [4/100], Training Loss: 0.006184336319849882
Epoch [4/100], Validation Loss: 0.006798737244132687
Model saved at epoch 4 with validation loss: 0.006798737244132687
Epoch [5/100], Training Loss: 0.0054549514425335
Epoch [5/100], Validation Loss: 0.007431440960701784
Epoch [6/100], Training Loss: 0.0048228091685727816
Epoch [6/100], Validation Loss: 0.006723169088224613
Model saved at epoch 6 with validation loss: 0.006723169088224613
Epoch [7/100], Training Loss: 0.0048604436141388195
Epoch [7/100], Validation Loss: 0.005937016141158279
Model sa

KeyboardInterrupt: 